In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import tensorflow as tf
from tensorflow.keras import models,layers

import matplotlib.pyplot as plt

import os

from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
im_size = 256
batch_size = 32
channel = 3

In [3]:
datagen = ImageDataGenerator(rescale=1./255,
                            horizontal_flip=True,vertical_flip=True,
                            zoom_range=0.2,rotation_range=30,
                            height_shift_range=0.2,width_shift_range=0.2,
                            validation_split=0.2)

In [4]:
traindata = datagen.flow_from_directory('../input/potato/Potato',
                                        class_mode='sparse',
                                        subset='training',
                                        target_size=(im_size,im_size))

In [5]:
valdata = datagen.flow_from_directory('../input/potato/Potato',
                                      class_mode='sparse',
                                      subset='validation',
                                      target_size=(im_size,im_size))

In [6]:
for im_batch,label_batch in traindata:
    print(im_batch.shape)
    break

In [7]:
class_names = list(traindata.class_indices)

class_names = dataset.class_names # from prev tf model

def get_train_test_splits(ds,train_split=0.7,test_split=0.15,val_split=0.15):
    
    train_size = int((len(ds)*train_split))
    val_size = int((len(ds)*test_split))
        
    test_ds = ds.skip(train_size).take(val_size)
    val_ds = ds.skip(train_size).skip(val_size)
    
    return train_ds,val_ds,test_ds

 tf.keras.Sequential==tf.keras.models.Sequential                        both same

In [8]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Conv2D,MaxPool2D,Flatten,Dropout,InputLayer
from tensorflow.keras.callbacks import EarlyStopping

In [9]:
input_shape = (im_size,im_size,channel)
input_shape

In [10]:
model = Sequential([
                    Conv2D(32,(3,3),activation='relu',input_shape=input_shape),
                    MaxPool2D(),
                    Conv2D(64,(3,3),activation='relu'),
                    MaxPool2D(),
                    Conv2D(64,(3,3),activation='relu'),
                    MaxPool2D(),
                    Conv2D(64,(3,3),activation='relu'),
                    MaxPool2D(),
                    Conv2D(64,(3,3),activation='relu'),                    
                    Flatten(),
                    Dense(3,activation='softmax')                    
                   ])

In [11]:
model.summary()

early_stopping = EarlyStopping( monitor='val_accuracy', 
                                patience=8,
                                min_delta=0.001,
                                mode='max')

In [12]:
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [13]:
epochs = 25
batch_size = 32

In [14]:
history = model.fit(traindata,
                    validation_data=valdata,
                    epochs=epochs,batch_size=batch_size) # add callbacks=[early_stopping] for early stop

In [15]:
# model.evaluate(test_ds)

In [16]:
history.params

In [17]:
history.history.keys()

In [18]:
loss = history.history['loss']
acc = history.history['accuracy']
val_loss = history.history['val_loss']
val_acc = history.history['val_accuracy']

In [19]:
plt.figure(figsize=(16,8))

plt.subplot(121)
plt.plot(range(epochs),loss,label = 'loss')
plt.plot(range(epochs),val_loss,label = 'Validation Loss')
plt.legend(loc='best')
plt.title('Training loss and accuracy')

plt.subplot(122)
plt.plot(range(epochs),acc,label = 'accuracy')
plt.plot(range(epochs),val_acc, label = 'Validation Accuracy')
plt.legend(loc='best')
plt.title('Val loss and accuracy')
plt.show()

In [20]:
for img_batch, label in valdata:
    
    im = img_batch[0]
    label = int(label[0])
    
    plt.imshow(im)
    plt.title(f'Actual Label {class_names[label]}')
    
    batch_pred = model.predict(img_batch)
    
    plt.suptitle(f'Predicted as : {class_names[np.argmax(batch_pred[0])]}')
                 
    break

In [21]:
model.save('modelVGG16/')

In [22]:
os.chdir(r'/kaggle/working')

In [23]:
from tensorflow.keras.preprocessing.image import img_to_array

In [24]:
def get_pred(model,img):
    img_array = img_to_array(img)
    img_batch = tf.expand_dims(img_array,0)   #create batch
    
    pred = model.predict(img_batch)
    
    predicted_class = class_names[np.argmax(pred[0])]
    
    confidence = np.max(pred[0])
    
    return predicted_class,confidence
    
    

In [25]:
#manually
#model_version = 1
#model.save(f'model/{model_version}')

In [26]:
import os 
# next time you run new model will be saved
model_version = max([int(i) for i in os.listdir('./model')])+1
model.save(f'model/{model_version}')